In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from clickhouse_driver import Client
from sqlalchemy import create_engine, Text, Integer, Float

In [2]:
periods_list = list()
end_date = datetime.date.today().replace(day=1)
start_date = end_date - relativedelta(months=13)
for i in range((end_date.year - start_date.year) * 12 + end_date.month - start_date.month):
    periods_list.append(str(start_date + relativedelta(months=i))[:7])
periods_list

['2022-02',
 '2022-03',
 '2022-04',
 '2022-05',
 '2022-06',
 '2022-07',
 '2022-08',
 '2022-09',
 '2022-10',
 '2022-11',
 '2022-12',
 '2023-01',
 '2023-02']

In [3]:
def get_clickhouse_client(host, user, password, db):
    client = Client(host=host,
                    user=user,
                    password=password,
                    database=db,
                    secure=True)
    return client

ch_host = ''
ch_user = 'elena'
ch_password = ''
ch_database = ''

client = get_clickhouse_client(ch_host, ch_user, ch_password, ch_database)

end_date = datetime.date.today().replace(day=1) - datetime.timedelta(1)
start_date = datetime.date.today().replace(day=1) - relativedelta(months=13)

print(start_date, '->', end_date)

subcategories = ['Борьба с несовершенствами',
                 'Уход для глаз и бровей',
                 'Уход для губ',
                 'Уход за лицом',
                 'Уход за руками',
                 'Уход за телом']

query = f'''
    with mpstats_cut as (
        select distinct
            mpstats.mpstats_id,
            mpstats.brand,
            mpstats.seller,
            mpstats.comments,
            mpstats.rating,
            mpstats.final_price,
            mpstats.sales,
            mpstats.revenue,
            mpstats."date",
            concat(
                splitByChar('-',  coalesce(mpstats."date",'-'))[1], '-',
                splitByChar('-',  coalesce(mpstats."date",'-'))[2]) as period
        from npddb.mps_wb_products mpstats
        where 1 = 1
            and mpstats."date" between '{str(start_date)}' and '{str(end_date)}'
            and splitByChar('/',  coalesce(mpstats.category,'-'))[1] = 'Красота'
            and splitByChar('/',  coalesce(mpstats.category,'-'))[2] = 'Уход за кожей'
            and splitByChar('/',  coalesce(mpstats.category,'-'))[3] in ({subcategories})
            and mpstats.brand <> ''
        )
    select mpstats_cut.brand,
        mpstats_cut.mpstats_id,
        mpstats_cut.period,
        sum(mpstats_cut.revenue) as revenue
    from mpstats_cut
    group by mpstats_cut.brand,
        mpstats_cut.mpstats_id,
        mpstats_cut.period
    having sum(mpstats_cut.revenue) <> 0
        '''

result, columns = client.execute(query, with_column_types=True)
df_init = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

2022-02-01 -> 2023-02-28


In [4]:
df = df_init.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139096 entries, 0 to 1139095
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   brand       1139096 non-null  object
 1   mpstats_id  1139096 non-null  int64 
 2   period      1139096 non-null  object
 3   revenue     1139096 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 34.8+ MB


In [5]:
df.sort_values(by=['brand','mpstats_id','period']).reset_index(drop=True).head(20)

,brand,mpstats_id,period,revenue
0,A'Pieu,37317831,2022-07,433
1,A'Pieu,38810617,2022-07,2445
2,A'Pieu,38810617,2022-08,468
3,A'Pieu,38812698,2022-07,423
4,A'Pieu,39015511,2022-07,712
5,A'Pieu,40014577,2022-07,1518
6,A'Pieu,41026320,2022-07,971
7,A'Pieu,41990480,2022-07,480
8,A'Pieu,42614900,2022-07,704
9,A'Pieu,43542749,2022-07,924


In [6]:
df['brand'] = [x.replace('\\','') for x in df['brand']]
diff_brands = df[['mpstats_id','brand']].groupby('mpstats_id').nunique().reset_index()
diff_brands = diff_brands[diff_brands['brand'] > 1].reset_index(drop=True)
diff_brands = diff_brands['mpstats_id'].to_list()
len(diff_brands)

6100

In [7]:
brands_rename = dict()
for i in diff_brands:
    df_i = df[df['mpstats_id'] == i].reset_index(drop=True)
    df_i = df_i.sort_values(by='period').drop_duplicates(subset='brand', keep='last')
    brands = df_i['brand'].to_list()
    for i in range(len(brands)-1):
        brands_rename[brands[i]] = brands[-1]
brands_rename

{'Sea of Spa': 'Black Pearl',
 'BIO MARINE': 'Sea of Spa',
 'KORA': 'КОРА',
 'BRADEX': 'Naomi Dead Sea Cosmetics',
 'Madis S.A.': 'HerbOlive',
 'Monalisa': 'BELLAGIO',
 'Synaa': 'Aasha Herbals',
 'ANDALOU': 'Andalou Naturals',
 'Nice Day': 'EcoLat',
 'EcoLat': 'ECOLAT Cosmetics',
 'DOMIX GREEN PROFESSIONAL': 'DOMIX',
 'Две Линии': 'ClioDerm',
 'товар12': 'Hons',
 'Wangprom Herbs': 'Wang Prom',
 'AsiaSpa': 'Тайский Секрет',
 'Joyberry': 'VARIANT.NAME',
 'Markell': 'МАРКЕЛЛ',
 'L Cosmetics': "L'Cosmetics",
 'Тайский Секрет': 'Тайскмй секрет',
 'Sante': 'Sante Naturkosmetik',
 'BIELITA': 'БЕЛИТА',
 'Vilenta': '7DAYS',
 'BARBERSHOP МУЖСКАЯ КОСМЕТИКА': 'ECOLAT BLACK',
 'ECOLAT BLACK': 'BARBERSHOP',
 'Удачная покупка': 'Touching Nature',
 'Romantic Color': 'Touching Nature',
 'NOTE': 'NOTE Cosmetique',
 'Бизорюк косметика': 'Бизорюк',
 'Бизорюк': 'Tambusun',
 'ODRY Style': 'ODRY STYLE Lab',
 'ODRYSTYLE Lab': 'ODRY STYLE Lab',
 'SYNERGETIC': 'Синергетик',
 'Synergetic.': 'SYNERGETIC',
 'Задар

In [8]:
for k, v in brands_rename.items():
    if v in list(brands_rename.keys()):
        df_v = df[df['brand'].isin([v, brands_rename[v]])][['brand','period']].drop_duplicates()
        df_v = df_v.sort_values(by='period').reset_index(drop=True)
        brands_rename[k] = df_v['brand'][len(df_v)-1]
        brands_rename[v] = df_v['brand'][len(df_v)-1]
brands_rename = {k:v for (k,v) in brands_rename.items() if k != v}
brands_rename

{'BIO MARINE': 'Sea of Spa',
 'KORA': 'КОРА',
 'BRADEX': 'Naomi Dead Sea Cosmetics',
 'Madis S.A.': 'HerbOlive',
 'Monalisa': 'BELLAGIO',
 'Synaa': 'Aasha Herbals',
 'ANDALOU': 'Andalou Naturals',
 'Nice Day': 'EcoLat',
 'DOMIX GREEN PROFESSIONAL': 'DOMIX',
 'Две Линии': 'ClioDerm',
 'Wangprom Herbs': 'Wang Prom',
 'AsiaSpa': 'Тайскмй секрет',
 'Joyberry': 'VARIANT.NAME',
 'Markell': 'МАРКЕЛЛ',
 'L Cosmetics': "L'Cosmetics",
 'Тайский Секрет': 'Тайскмй секрет',
 'Sante': 'Sante Naturkosmetik',
 'Vilenta': '7DAYS',
 'BARBERSHOP МУЖСКАЯ КОСМЕТИКА': 'BARBERSHOP',
 'ECOLAT BLACK': 'BARBERSHOP',
 'Удачная покупка': 'Touching Nature',
 'Romantic Color': 'Touching Nature',
 'NOTE': 'NOTE Cosmetique',
 'Бизорюк косметика': 'Tambusun',
 'Бизорюк': 'Tambusun',
 'ODRY Style': 'ODRY STYLE Lab',
 'ODRYSTYLE Lab': 'ODRY STYLE Lab',
 'Synergetic.': 'SYNERGETIC',
 'Задари': 'SAS',
 'Eva': 'Невский банщик',
 'ВЕСНА': 'ПКК Весна',
 'BIELITA Professional': 'Витэкс',
 "L'Sanic": 'L.Sanic',
 'DIVА': 'NO NA

In [9]:
len(brands_rename)

1832

In [10]:
for k in brands_rename.keys():
    df['brand'] = df['brand'].replace(k, brands_rename[k])
df = df[['brand','period','revenue']].groupby(['brand','period']).sum().reset_index()
df

,brand,period,revenue
0,Ai-Naz cosmeticus,2023-02,1000
1,ПЕРВАЯ ЛАСТОЧКА,2022-08,800
2,ПЕРВАЯ ЛАСТОЧКА,2022-09,590
3,ПЕРВАЯ ЛАСТОЧКА,2022-10,3540
4,ПЕРВАЯ ЛАСТОЧКА,2022-12,3400
...,...,...,...
122877,⭐️ Mega Star,2022-12,72588
122878,⭐️ Mega Star,2023-01,136137
122879,⭐️ Mega Star,2023-02,81750
122880,藥店,2023-01,384


In [11]:
brands = list(df[(df['period'] == periods_list[-1]) & (df['revenue'] > 2000000)]['brand'].unique())
for k, v in brands_rename.items():
    if v in brands:
        brands.append(k)
brands = list(set(brands))
len(brands)

850

In [12]:
class ReplaceDoubleQuotes(str):
    def __repr__(self):
        return ''.join(("'", super().__repr__()[1:-1], "'"))
    
brands = [x.replace("'", "''") for x in brands]
brands = [ReplaceDoubleQuotes(x) for x in brands]
brands

['Image',
 'BELWEDER',
 'Свежая нота',
 'Morozov Tattoo',
 'Eskin',
 'facefit.',
 'ROSARIUM',
 'Сила растений',
 'Wow Results',
 'NATUROTEKA',
 'Cos De Baha',
 'STARBERRY',
 'Lady&Lina',
 'Dr. Healux',
 'ZOZU TM',
 'FOREVER-Y',
 '24K GOLD SKIN CARE',
 'Faberlic',
 'ICHTHYONELLA',
 'ARAVIA Organic',
 'RENI',
 'INKI',
 'BEAUTIFIC',
 'GRASS',
 'Beauty Club',
 'Мирролла',
 'Невская Косметика.',
 'SAIAN',
 'Vasilisa Beauty',
 'SHINE BRIGHT ✨',
 'Natura Sibericа',
 'HCHANA',
 'SYSTEO',
 'Пенка с щеткой',
 'BESTLAND',
 'CAMAY',
 'Mustela',
 'The VIRgin Beauty',
 'DOMIX GREEN PROFESSIONAL',
 'Бизорюк косметика',
 'KAMCHATKA',
 'MEDI-PEEL Cosmetics',
 'Zozu',
 'Gift Station',
 'Enjoy yourself',
 'Beauty Malinka',
 'Meditime',
 'PixelTap Beauty',
 'Виноград/Farmstay',
 'Genera',
 'Thai Traditions',
 'ARAVIA\xa0Professional',
 'Dr. Sea',
 'Beautytreea',
 'Fessco',
 'POITEAG',
 'MPBuy',
 'Biore',
 'L''Oreal Paris',
 'Бьюти бокс',
 'Ali',
 'Royal Samples',
 'SKUINA',
 'FARIMS skins',
 'MEZOLUX',
 '

In [13]:
query = f'''
select distinct
    mpstats.mpstats_id,
    mpstats.brand,
    mpstats.seller,
    mpstats.comments,
    mpstats.rating,
    mpstats.final_price,
    mpstats.sales,
    mpstats.revenue,
    mpstats."date",
    concat(
        splitByChar('-',  coalesce(mpstats."date",'-'))[1], '-',
        splitByChar('-',  coalesce(mpstats."date",'-'))[2]) as period
from npddb.mps_wb_products mpstats
where 1 = 1
    and (mpstats."date" between '{str(start_date)}' and '{str(end_date)}')
    and splitByChar('/',  coalesce(mpstats.category,'-'))[1] = 'Красота'
    and splitByChar('/',  coalesce(mpstats.category,'-'))[2] = 'Уход за кожей'
    and splitByChar('/',  coalesce(mpstats.category,'-'))[3] in ({subcategories})
    and mpstats.brand in ({brands})
        '''

result, columns = client.execute(query, with_column_types=True)
df_init = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

In [14]:
df = df_init.copy()
df.shape

(16130962, 10)

In [15]:
df['mpstats_id'] = df['mpstats_id'].astype(str)
df['date'] = [str(x)[:10] for x in df['date']]
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16130962 entries, 0 to 16130961
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   mpstats_id   object        
 1   brand        object        
 2   seller       object        
 3   comments     int64         
 4   rating       int64         
 5   final_price  int64         
 6   sales        int64         
 7   revenue      int64         
 8   date         datetime64[ns]
 9   period       object        
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 1.2+ GB


In [16]:
brand_rename = pd.DataFrame.from_dict(brands_rename, orient='index').reset_index()
brand_rename = brand_rename.rename(columns={'index': 'brand_from', 0: 'brand_to'})
brand_rename

,brand_from,brand_to
0,BIO MARINE,Sea of Spa
1,KORA,КОРА
2,BRADEX,Naomi Dead Sea Cosmetics
3,Madis S.A.,HerbOlive
4,Monalisa,BELLAGIO
...,...,...
1827,Aloe Beauty,Чижов&Beauty
1828,Carren,ELIZAVECCA
1829,Lucas Cosmetics,Chiatoprelest
1830,Tooba,Fair & Lovely


In [17]:
df = df.merge(brand_rename, how='left', left_on='brand', right_on='brand_from')
df['brand'] = np.where(df['brand_from'].isna(), df['brand'], df['brand_to'])
df = df.drop(['brand_from','brand_to'], axis=1)
df.head(5)

,mpstats_id,brand,seller,comments,rating,final_price,sales,revenue,date,period
0,52256366,cosmo beauty,Гукасян Цолак Геворгович,1,5,769,0,0,2022-07-11,2022-07
1,52276965,THE ORDINARY,ИП Ak Sale,1,5,2040,0,0,2022-07-11,2022-07
2,52286705,LANBENA,ИП Пестрикова С В,0,0,346,0,0,2022-07-11,2022-07
3,52382811,LANBENA,ИП Захидова Т В,24,4,394,2,788,2022-07-11,2022-07
4,52397410,Витэкс,ИП Титовцов Андрей Васильевич,20,5,372,0,0,2022-07-11,2022-07


In [18]:
df['seller'] = [x.replace('\\','') for x in df['seller']]
diff_sellers = df[['mpstats_id','seller']].groupby('mpstats_id').nunique().reset_index()
diff_sellers = diff_sellers[diff_sellers['seller'] > 1].reset_index(drop=True)
diff_sellers = diff_sellers['mpstats_id'].to_list()
len(diff_sellers)

15209

In [19]:
sellers_rename = dict()
for i in diff_sellers:
    df_i = df[df['mpstats_id'] == i].reset_index(drop=True)
    df_i = df_i.sort_values(by='date').drop_duplicates(subset='seller', keep='last')
    sellers = df_i['seller'].to_list()
    for i in range(len(sellers)-1):
        sellers_rename[sellers[i]] = sellers[-1]
sellers_rename

{'': 'ИП Садабаева',
 'Пирогов Владислав Алексеевич': 'ИП Пирогов В А',
 'ОАО Единая Европа-Холдинг': 'ООО Вайлдберриз',
 'ДК Казына': 'ИП Дк Казына',
 'ИП Илюхин': 'Космоэль',
 'ООО ЭЛР': 'ООО Эстилаб РУС',
 'ИП Магомедов Р Г': 'Магомедов Расул',
 'ООО Firmach': 'Магомедов Расул',
 'ИП Cosmetolog': 'ИП Prof Cosmetics',
 'ИП Aisha': 'Aisha Shop',
 'ООО Кларанс': 'ООО Вайлдберриз',
 'ИП Бабаева И О': 'Royal Diamond',
 'ИП Калдар': '4You',
 'Feliz': 'Suerte',
 'ИП Морару Алина Ивановна': 'Moraru',
 'Эсманов Илья Петрович': 'ИП Эсманов Илья Петрович',
 'ИП Ашимханова': 'Only Store',
 'ИП Павлова Екатерина Сергеевна': 'Ketrincosmetics',
 'ИП Драгун Валентин': 'ИП Драгун В',
 'ИП Левит М Д': 'Mpbuy',
 'Чистый мир': 'Queen Cosmetics',
 'ИП Алимкулов Н А': 'ИП Н А',
 'ИП Гараев З Х': 'Shamsa',
 'ООО Вайлдберриз': '',
 'ИП Тараканов Иван Андреевич': 'Crystal',
 'ИП Емуранов': 'Swoosh',
 'ИП Жрснбек': 'ИП Журсинбек',
 'ИП Aminad': 'ИП Laviva',
 'ИП Жораев Д Г': 'OOO Жораев Д. Г',
 'Ignt Shop': 

In [20]:
sellers_rename

{'': 'ИП Садабаева',
 'Пирогов Владислав Алексеевич': 'ИП Пирогов В А',
 'ОАО Единая Европа-Холдинг': 'ООО Вайлдберриз',
 'ДК Казына': 'ИП Дк Казына',
 'ИП Илюхин': 'Космоэль',
 'ООО ЭЛР': 'ООО Эстилаб РУС',
 'ИП Магомедов Р Г': 'Магомедов Расул',
 'ООО Firmach': 'Магомедов Расул',
 'ИП Cosmetolog': 'ИП Prof Cosmetics',
 'ИП Aisha': 'Aisha Shop',
 'ООО Кларанс': 'ООО Вайлдберриз',
 'ИП Бабаева И О': 'Royal Diamond',
 'ИП Калдар': '4You',
 'Feliz': 'Suerte',
 'ИП Морару Алина Ивановна': 'Moraru',
 'Эсманов Илья Петрович': 'ИП Эсманов Илья Петрович',
 'ИП Ашимханова': 'Only Store',
 'ИП Павлова Екатерина Сергеевна': 'Ketrincosmetics',
 'ИП Драгун Валентин': 'ИП Драгун В',
 'ИП Левит М Д': 'Mpbuy',
 'Чистый мир': 'Queen Cosmetics',
 'ИП Алимкулов Н А': 'ИП Н А',
 'ИП Гараев З Х': 'Shamsa',
 'ООО Вайлдберриз': '',
 'ИП Тараканов Иван Андреевич': 'Crystal',
 'ИП Емуранов': 'Swoosh',
 'ИП Жрснбек': 'ИП Журсинбек',
 'ИП Aminad': 'ИП Laviva',
 'ИП Жораев Д Г': 'OOO Жораев Д. Г',
 'Ignt Shop': 

In [21]:
for k, v in sellers_rename.items():
    if v in list(sellers_rename.keys()):
        df_v = df[df['seller'].isin([v, sellers_rename[v]])][['seller','date']].drop_duplicates()
        df_v = df_v.sort_values(by='date').reset_index(drop=True)
        sellers_rename[k] = df_v['seller'][len(df_v)-1]
        sellers_rename[v] = df_v['seller'][len(df_v)-1]
sellers_rename = {k:v for (k,v) in sellers_rename.items()
                  if k != v and 'вайлдберриз' not in k.lower()}
sellers_rename

{'': 'ИП Садабаева',
 'Пирогов Владислав Алексеевич': 'ИП Пирогов В А',
 'ОАО Единая Европа-Холдинг': '',
 'ДК Казына': 'ИП Дк Казына',
 'ИП Илюхин': 'Космоэль',
 'ООО ЭЛР': 'ООО Эстилаб РУС',
 'ИП Магомедов Р Г': 'Магомедов Расул',
 'ООО Firmach': 'Магомедов Расул',
 'ИП Cosmetolog': 'ИП Prof Cosmetics',
 'ИП Aisha': 'Aisha Shop',
 'ООО Кларанс': '',
 'ИП Бабаева И О': 'Royal Diamond',
 'ИП Калдар': '4You',
 'Feliz': 'Suerte',
 'ИП Морару Алина Ивановна': 'Moraru',
 'Эсманов Илья Петрович': 'ИП Эсманов Илья Петрович',
 'ИП Ашимханова': 'Only Store',
 'ИП Павлова Екатерина Сергеевна': 'Ketrincosmetics',
 'ИП Драгун Валентин': 'ИП Драгун В',
 'ИП Левит М Д': 'Mpbuy',
 'Чистый мир': 'Queen Cosmetics',
 'ИП Алимкулов Н А': 'ИП Н А',
 'ИП Гараев З Х': 'Shamsa',
 'ИП Тараканов Иван Андреевич': 'Crystal',
 'ИП Емуранов': 'Swoosh',
 'ИП Жрснбек': 'ИП Журсинбек',
 'ИП Aminad': 'ИП Laviva',
 'ИП Жораев Д Г': 'OOO Жораев Д. Г',
 'Ignt Shop': 'Venky Store',
 'ИП Шевцов Антон Александрович': 'ИП Ш

In [22]:
sellers_rename = pd.DataFrame.from_dict(sellers_rename, orient='index').reset_index()
sellers_rename = sellers_rename.rename(columns={'index': 'seller_from', 0: 'seller_to'})
sellers_rename

,seller_from,seller_to
0,,ИП Садабаева
1,Пирогов Владислав Алексеевич,ИП Пирогов В А
2,ОАО Единая Европа-Холдинг,
3,ДК Казына,ИП Дк Казына
4,ИП Илюхин,Космоэль
...,...,...
1166,ИП Астафуров Вю,ИП Астафуров В Ю
1167,"Асым Г,Е",Gulasule Kassym
1168,ИП Айтенов,Comfortex
1169,ИП Zetechnology,High Speed


In [23]:
df = df.merge(sellers_rename, how='left', left_on='seller', right_on='seller_from')
df['seller'] = np.where(df['seller_from'].isna(), df['seller'], df['seller_to'])
df = df.drop(['seller_from','seller_to'], axis=1)
df.head(5)

,mpstats_id,brand,seller,comments,rating,final_price,sales,revenue,date,period
0,52256366,cosmo beauty,Гукасян Цолак Геворгович,1,5,769,0,0,2022-07-11,2022-07
1,52276965,THE ORDINARY,ИП Ak Sale,1,5,2040,0,0,2022-07-11,2022-07
2,52286705,LANBENA,ИП Пестрикова С В,0,0,346,0,0,2022-07-11,2022-07
3,52382811,LANBENA,ИП Захидова Т В,24,4,394,2,788,2022-07-11,2022-07
4,52397410,Витэкс,ИП Титовцов Андрей Васильевич,20,5,372,0,0,2022-07-11,2022-07


In [24]:
# df.head(10)

In [25]:
# pivot_revenue = df.pivot_table(values=['revenue'],
#                                index=['brand'],
#                                columns=['period'],
#                                aggfunc='sum',
#                                fill_value=0)
# pivot_revenue.columns = list(map(''.join, pivot_revenue.columns))
# pivot_revenue.columns = periods_list
# pivot_revenue.insert(0, 'metrics', 'Выручка')
# pivot_revenue.shape

In [26]:
# pivot_price = df.pivot_table(values=['final_price'],
#                              index=['brand'],
#                              columns=['period'],
#                              aggfunc='mean',
#                              fill_value=0)
# pivot_price.columns = list(map(''.join, pivot_price.columns))
# pivot_price.columns = periods_list
# pivot_price.insert(0, 'metrics', 'Средняя цена')
# pivot_price.shape

In [27]:
# pivot_sku = df.pivot_table(values=['mpstats_id'],
#                            index=['brand'],
#                            columns=['period'],
#                            aggfunc='nunique',
#                            fill_value=0)
# pivot_sku.columns = list(map(''.join, pivot_sku.columns))
# pivot_sku.columns = periods_list
# pivot_sku.insert(0, 'metrics', 'Количество SKU')
# pivot_sku.shape

In [28]:
# df_res = pd.concat([pivot_revenue, pivot_price, pivot_sku], axis=0)
# df_res.shape

In [29]:
# df_res.tail(5)

In [30]:
# df_res = df_res.reset_index()
# df_res.to_excel('Динамика по брендам.xlsx', index=False)

In [31]:
main_table = df[df['period'] == periods_list[-1]].groupby(['brand']).agg(
    revenue = pd.NamedAgg('revenue', 'sum'),  # выручка (последний месяц)
    avg_price = pd.NamedAgg('final_price', 'mean'),  # средняя цена (последний месяц)
    sku_num = pd.NamedAgg('mpstats_id', 'nunique'),  # общее число sku (последний месяц)
    sellers = pd.NamedAgg('seller', 'nunique')  # количество продавцов (последний месяц)
    ).reset_index()
main_table['sku_rev'] = main_table['revenue'] / main_table['sku_num']
main_table

,brand,revenue,avg_price,sku_num,sellers,sku_rev
0,7DAYS,15430461,549.901152,393,19,39263.259542
1,A.H.C.,3180058,1117.366426,190,83,16737.147368
2,AEVIT BY LIBREDERM,4581487,412.546453,59,6,77652.322034
3,AFRODITA STYLE,2826075,9687.855172,6,1,471012.500000
4,ARAVIA Laboratories,50440746,1010.940086,173,24,291565.005780
...,...,...,...,...,...,...
373,Фитокосметик,3500286,354.584822,245,13,14286.881633
374,Циновит,2392342,1164.634074,35,10,68352.628571
375,ЧИСТАЯ ЛИНИЯ,6463515,492.551029,525,75,12311.457143
376,Чижов&Beauty,27150404,393.843345,144,27,188544.472222


In [32]:
main_table = main_table[main_table['revenue'] > 2000000].reset_index(drop=True)

In [33]:
# число sku с продажами (последний месяц)
sku_with_sales = df[(df['period'] == periods_list[-1])].groupby(['brand','mpstats_id']).agg(
    sales = pd.NamedAgg('sales', 'sum')).reset_index()
sku_with_sales = sku_with_sales[sku_with_sales['sales'] != 0][['brand','mpstats_id']].groupby(['brand']).count()
sku_with_sales = sku_with_sales.rename(columns={'mpstats_id': 'sku_with_sales_num'}).reset_index()
sku_with_sales

,brand,sku_with_sales_num
0,7DAYS,286
1,A.H.C.,75
2,AEVIT BY LIBREDERM,45
3,AFRODITA STYLE,6
4,ARAVIA Laboratories,95
...,...,...
372,Фитокосметик,162
373,Циновит,13
374,ЧИСТАЯ ЛИНИЯ,109
375,Чижов&Beauty,116


In [34]:
main_table = main_table.merge(sku_with_sales, how='left', on='brand')
main_table['sku_with_sales'] = main_table['sku_with_sales_num'] / main_table['sku_num']  # % sku с продажами
main_table

,brand,revenue,avg_price,sku_num,sellers,sku_rev,sku_with_sales_num,sku_with_sales
0,7DAYS,15430461,549.901152,393,19,39263.259542,286,0.727735
1,A.H.C.,3180058,1117.366426,190,83,16737.147368,75,0.394737
2,AEVIT BY LIBREDERM,4581487,412.546453,59,6,77652.322034,45,0.762712
3,AFRODITA STYLE,2826075,9687.855172,6,1,471012.500000,6,1.000000
4,ARAVIA Laboratories,50440746,1010.940086,173,24,291565.005780,95,0.549133
...,...,...,...,...,...,...,...,...
369,Фитокосметик,3500286,354.584822,245,13,14286.881633,162,0.661224
370,Циновит,2392342,1164.634074,35,10,68352.628571,13,0.371429
371,ЧИСТАЯ ЛИНИЯ,6463515,492.551029,525,75,12311.457143,109,0.207619
372,Чижов&Beauty,27150404,393.843345,144,27,188544.472222,116,0.805556


In [35]:
# число sku в предыдущем месяце
sku_num_prev = df[df['period'] == periods_list[-2]].groupby(['brand']).agg(
    sku_num_prev = pd.NamedAgg('mpstats_id', 'nunique')).reset_index()
sku_num_prev

,brand,sku_num_prev
0,7DAYS,370
1,A.H.C.,179
2,AEVIT BY LIBREDERM,52
3,AFRODITA STYLE,6
4,ARAVIA Laboratories,173
...,...,...
373,Фитокосметик,322
374,Циновит,34
375,ЧИСТАЯ ЛИНИЯ,475
376,Чижов&Beauty,168


In [36]:
main_table = main_table.merge(sku_num_prev, how='left', on='brand').fillna(0)
main_table['sku_new'] = main_table['sku_num'] - main_table['sku_num_prev']
main_table = main_table.drop(['sku_num_prev'], axis=1)
main_table

,brand,revenue,avg_price,sku_num,sellers,sku_rev,sku_with_sales_num,sku_with_sales,sku_new
0,7DAYS,15430461,549.901152,393,19,39263.259542,286,0.727735,23
1,A.H.C.,3180058,1117.366426,190,83,16737.147368,75,0.394737,11
2,AEVIT BY LIBREDERM,4581487,412.546453,59,6,77652.322034,45,0.762712,7
3,AFRODITA STYLE,2826075,9687.855172,6,1,471012.500000,6,1.000000,0
4,ARAVIA Laboratories,50440746,1010.940086,173,24,291565.005780,95,0.549133,0
...,...,...,...,...,...,...,...,...,...
369,Фитокосметик,3500286,354.584822,245,13,14286.881633,162,0.661224,-77
370,Циновит,2392342,1164.634074,35,10,68352.628571,13,0.371429,1
371,ЧИСТАЯ ЛИНИЯ,6463515,492.551029,525,75,12311.457143,109,0.207619,50
372,Чижов&Beauty,27150404,393.843345,144,27,188544.472222,116,0.805556,-24


In [37]:
products_rating = df[['brand','mpstats_id','period','date','comments','rating']]
products_rating = products_rating.sort_values(['brand','mpstats_id','date'])
products_rating = products_rating.drop_duplicates(['brand','mpstats_id','period'], keep='last')
products_rating = products_rating.drop(['date'], axis=1).reset_index(drop=True)
products_rating.head(10)

,brand,mpstats_id,period,comments,rating
0,7DAYS,100122324,2022-09,0,0
1,7DAYS,100122324,2022-10,0,0
2,7DAYS,100122324,2022-11,0,0
3,7DAYS,100122838,2022-09,0,0
4,7DAYS,100122838,2022-10,0,0
5,7DAYS,100122838,2022-11,0,0
6,7DAYS,100322864,2022-09,163,5
7,7DAYS,100322864,2022-10,206,5
8,7DAYS,100322864,2022-11,225,5
9,7DAYS,100322864,2022-12,373,5


In [38]:
# средний рейтинг бренда (последний месяц)
# количество новых отзывов (последний месяц)

brand_rating = products_rating[products_rating['period'] == periods_list[-1]].groupby(['brand']).agg(
    avg_rating = pd.NamedAgg('rating', 'mean')).reset_index()

new_comments = products_rating[products_rating['period'] == periods_list[-1]][['brand','mpstats_id','comments']].merge(
       products_rating[products_rating['period'] == periods_list[-2]][['brand','mpstats_id','comments']],
        how='inner', on=['brand','mpstats_id'])

new_comments = new_comments.groupby(['brand']).agg(
    comments_x = pd.NamedAgg('comments_x', 'sum'),
    comments_y = pd.NamedAgg('comments_y', 'sum')).reset_index()
new_comments['new_comments'] = new_comments['comments_x'] - new_comments['comments_y']

brand_rating = brand_rating.merge(new_comments[['brand','new_comments']], how='left', on='brand')
brand_rating

,brand,avg_rating,new_comments
0,7DAYS,4.142494,7602
1,A.H.C.,2.310526,360
2,AEVIT BY LIBREDERM,4.169492,692
3,AFRODITA STYLE,4.833333,38
4,ARAVIA Laboratories,2.369942,2593
...,...,...,...
373,Фитокосметик,3.293878,853
374,Циновит,1.914286,113
375,ЧИСТАЯ ЛИНИЯ,1.480000,2431
376,Чижов&Beauty,3.888889,9666


In [39]:
main_table = main_table.merge(brand_rating, how='inner', on='brand')
main_table

,brand,revenue,avg_price,sku_num,sellers,sku_rev,sku_with_sales_num,sku_with_sales,sku_new,avg_rating,new_comments
0,7DAYS,15430461,549.901152,393,19,39263.259542,286,0.727735,23,4.142494,7602
1,A.H.C.,3180058,1117.366426,190,83,16737.147368,75,0.394737,11,2.310526,360
2,AEVIT BY LIBREDERM,4581487,412.546453,59,6,77652.322034,45,0.762712,7,4.169492,692
3,AFRODITA STYLE,2826075,9687.855172,6,1,471012.500000,6,1.000000,0,4.833333,38
4,ARAVIA Laboratories,50440746,1010.940086,173,24,291565.005780,95,0.549133,0,2.369942,2593
...,...,...,...,...,...,...,...,...,...,...,...
369,Фитокосметик,3500286,354.584822,245,13,14286.881633,162,0.661224,-77,3.293878,853
370,Циновит,2392342,1164.634074,35,10,68352.628571,13,0.371429,1,1.914286,113
371,ЧИСТАЯ ЛИНИЯ,6463515,492.551029,525,75,12311.457143,109,0.207619,50,1.480000,2431
372,Чижов&Beauty,27150404,393.843345,144,27,188544.472222,116,0.805556,-24,3.888889,9666


In [40]:
dynamics = pd.DataFrame({'brand': [],
                        'revenue_period': [],
                        'avg_price_period': [],
                        'sku_num_period': [],
                        'sku_sales_period': [],
                        'sku_rev_period': [],
                        'avg_rating_period': [],
                        'period': []})

for period in [0,6,9]:
    
    df_period = df[df['period'] == periods_list[period]].groupby(['brand']).agg(
        revenue_period = pd.NamedAgg('revenue', 'sum'),
        avg_price_period = pd.NamedAgg('final_price', 'mean'),
        sku_num_period = pd.NamedAgg('mpstats_id', 'nunique')).reset_index()
    df_period['sku_rev_period'] = df_period['revenue_period'] / df_period['sku_num_period']
    
    brand_rating = products_rating[products_rating['period'] == periods_list[period]].groupby(['brand']).agg(
        avg_rating_period = pd.NamedAgg('rating', 'mean')).reset_index()
    df_period = df_period.merge(brand_rating, how='left', on='brand').fillna(0)  # средний рейтинг бренда
    
    sku_with_sales = df[(df['period'] == periods_list[period])].groupby(['brand','mpstats_id']).agg(
        sales = pd.NamedAgg('sales', 'sum')).reset_index()
    sku_with_sales = sku_with_sales[sku_with_sales['sales'] != 0][['brand','mpstats_id']].groupby(['brand']).count()
    sku_with_sales = sku_with_sales.rename(columns={'mpstats_id': 'sku_sales_period'}).reset_index()
    
    df_period = df_period.merge(sku_with_sales, how='left', on='brand').fillna(0)
    df_period['period'] = 12 - period
    
    dynamics = dynamics.append(df_period, ignore_index=True)
    
dynamics

,brand,revenue_period,avg_price_period,sku_num_period,sku_sales_period,sku_rev_period,avg_rating_period,period
0,7DAYS,26684838.0,320.593352,314.0,311.0,84983.560510,4.299363,12.0
1,A.H.C.,1143549.0,1530.572474,72.0,35.0,15882.625000,2.722222,12.0
2,AEVIT BY LIBREDERM,1632922.0,297.051593,37.0,36.0,44133.027027,4.324324,12.0
3,ARAVIA Laboratories,24931523.0,599.783677,61.0,61.0,408713.491803,4.393443,12.0
4,ARAVIA Organic,13112048.0,812.883087,42.0,42.0,312191.619048,4.476190,12.0
...,...,...,...,...,...,...,...,...
1059,Фитокосметик,1423498.0,322.440238,317.0,182.0,4490.529968,2.466877,3.0
1060,Циновит,274350.0,1359.250608,29.0,10.0,9460.344828,1.896552,3.0
1061,ЧИСТАЯ ЛИНИЯ,5685750.0,515.606686,435.0,87.0,13070.689655,1.234483,3.0
1062,Чижов&Beauty,13344197.0,365.089463,187.0,132.0,71359.342246,3.529412,3.0


In [41]:
dynamics[dynamics['brand'] == '3W Clinic']

,brand,revenue_period,avg_price_period,sku_num_period,sku_sales_period,sku_rev_period,avg_rating_period,period


In [42]:
dynamics['period'].unique()

array([12.,  6.,  3.])

In [43]:
new_brands = main_table[~main_table['brand'].isin(dynamics['brand'])][['brand']].reset_index(drop=True)
new_brands = new_brands.merge(pd.DataFrame(dynamics['period'].unique()), how='cross')
new_brands = new_brands.rename(columns={0: 'period'})
dynamics = dynamics.append(new_brands, ignore_index=True)
dynamics

,brand,revenue_period,avg_price_period,sku_num_period,sku_sales_period,sku_rev_period,avg_rating_period,period
0,7DAYS,26684838.0,320.593352,314.0,311.0,84983.560510,4.299363,12.0
1,A.H.C.,1143549.0,1530.572474,72.0,35.0,15882.625000,2.722222,12.0
2,AEVIT BY LIBREDERM,1632922.0,297.051593,37.0,36.0,44133.027027,4.324324,12.0
3,ARAVIA Laboratories,24931523.0,599.783677,61.0,61.0,408713.491803,4.393443,12.0
4,ARAVIA Organic,13112048.0,812.883087,42.0,42.0,312191.619048,4.476190,12.0
...,...,...,...,...,...,...,...,...
1074,Магеря Косметик,NaN,NaN,NaN,NaN,NaN,NaN,6.0
1075,Магеря Косметик,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1076,Миноксидил Киркланд 5%,NaN,NaN,NaN,NaN,NaN,NaN,12.0
1077,Миноксидил Киркланд 5%,NaN,NaN,NaN,NaN,NaN,NaN,6.0


In [44]:
main_table = main_table.merge(dynamics, how='left', on='brand').fillna(0)
main_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068 entries, 0 to 1067
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               1068 non-null   object 
 1   revenue             1068 non-null   int64  
 2   avg_price           1068 non-null   float64
 3   sku_num             1068 non-null   int64  
 4   sellers             1068 non-null   int64  
 5   sku_rev             1068 non-null   float64
 6   sku_with_sales_num  1068 non-null   int64  
 7   sku_with_sales      1068 non-null   float64
 8   sku_new             1068 non-null   int64  
 9   avg_rating          1068 non-null   float64
 10  new_comments        1068 non-null   int64  
 11  revenue_period      1068 non-null   float64
 12  avg_price_period    1068 non-null   float64
 13  sku_num_period      1068 non-null   float64
 14  sku_sales_period    1068 non-null   float64
 15  sku_rev_period      1068 non-null   float64
 16  avg_ra

In [45]:
main_table['rev_g'] = main_table['revenue'] / main_table['revenue_period'] - 1
# main_table['sku_g'] = main_table['sku_num'] / main_table['sku_num_period'] - 1
main_table['sku_g'] = main_table['sku_num'] - main_table['sku_num_period']
# main_table['sku_s_g'] = main_table['sku_with_sales_num'] / main_table['sku_sales_period'] - 1
main_table['sku_s_g'] = main_table['sku_with_sales_num'] - main_table['sku_sales_period']
main_table['sku_rev_g'] = main_table['sku_rev'] / main_table['sku_rev_period'] - 1
main_table['price_g'] = main_table['avg_price'] / main_table['avg_price_period'] - 1
# main_table['rating_g'] = main_table['avg_rating'] / main_table['avg_rating_period'] - 1
main_table['rating_g'] = main_table['avg_rating'] - main_table['avg_rating_period']

main_table = main_table.drop(['revenue_period',
                              'sku_num_period',
                              'sku_sales_period',
                              'sku_rev_period',
                              'avg_price_period',
                              'avg_rating_period'
                             ], axis=1)
main_table = main_table.replace([np.inf, -np.inf], np.nan).fillna(0)
main_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068 entries, 0 to 1067
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               1068 non-null   object 
 1   revenue             1068 non-null   int64  
 2   avg_price           1068 non-null   float64
 3   sku_num             1068 non-null   int64  
 4   sellers             1068 non-null   int64  
 5   sku_rev             1068 non-null   float64
 6   sku_with_sales_num  1068 non-null   int64  
 7   sku_with_sales      1068 non-null   float64
 8   sku_new             1068 non-null   int64  
 9   avg_rating          1068 non-null   float64
 10  new_comments        1068 non-null   int64  
 11  period              1068 non-null   float64
 12  rev_g               1068 non-null   float64
 13  sku_g               1068 non-null   float64
 14  sku_s_g             1068 non-null   float64
 15  sku_rev_g           1068 non-null   float64
 16  price_

In [46]:
connection_string = 'postgresql://elena:<j,hlj,h1@rc1b-bojd020c96w0gyci.mdb.yandexcloud.net:6432/dwh'
engine = create_engine(connection_string)
main_table.to_sql(name='market_by_brands',
                  con=engine,
                  if_exists='replace',
                  schema='sa',
                  chunksize=10024,
                  index=False,
                  method='multi',
                  dtype={
                      'brand': Text(),
                      'revenue': Float(),
                      'avg_price': Float(),
                      'sku_num': Integer(),
                      'sellers': Integer(),
                      'sku_rev': Float(),
                      'sku_with_sales': Float(),
                      'sku_new': Integer(),
                      'avg_rating': Float(),
                      'new_comments': Integer(),
                      'period': Integer(),
                      'rev_g': Float(),
                      'sku_g': Integer(),
                      'sku_s_g': Integer(),
                      'sku_rev_g': Float(),
                      'price_g': Float(),
                      'rating_g': Float()
                  })
engine.dispose()

In [47]:
main_table

,brand,revenue,avg_price,sku_num,sellers,sku_rev,sku_with_sales_num,sku_with_sales,sku_new,avg_rating,new_comments,period,rev_g,sku_g,sku_s_g,sku_rev_g,price_g,rating_g
0,7DAYS,15430461,549.901152,393,19,39263.259542,286,0.727735,23,4.142494,7602,12.0,-0.421752,79.0,-25.0,-0.537990,0.715261,-0.156869
1,7DAYS,15430461,549.901152,393,19,39263.259542,286,0.727735,23,4.142494,7602,6.0,1.822425,-7.0,-13.0,1.872697,0.357986,0.472494
2,7DAYS,15430461,549.901152,393,19,39263.259542,286,0.727735,23,4.142494,7602,3.0,1.183185,-33.0,-28.0,1.366506,-0.064406,0.248127
3,A.H.C.,3180058,1117.366426,190,83,16737.147368,75,0.394737,11,2.310526,360,12.0,1.780867,118.0,40.0,0.053802,-0.269968,-0.411696
4,A.H.C.,3180058,1117.366426,190,83,16737.147368,75,0.394737,11,2.310526,360,6.0,1.777761,18.0,20.0,1.514604,-0.209649,0.455875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,Чижов&Beauty,27150404,393.843345,144,27,188544.472222,116,0.805556,-24,3.888889,9666,6.0,1.391016,-144.0,-48.0,3.782032,-0.139667,1.083333
1064,Чижов&Beauty,27150404,393.843345,144,27,188544.472222,116,0.805556,-24,3.888889,9666,3.0,1.034623,-43.0,-16.0,1.642183,0.078758,0.359477
1065,Шелковица,3048153,1163.419825,36,1,84670.916667,34,0.944444,5,4.916667,688,12.0,0.348336,24.0,22.0,-0.550555,-0.080217,-0.083333
1066,Шелковица,3048153,1163.419825,36,1,84670.916667,34,0.944444,5,4.916667,688,6.0,0.830306,23.0,21.0,-0.339056,0.039553,0.301282
